# ⚡ Week 5: FastAPI Backend

**Learning Objectives:**
1. Build async REST APIs with FastAPI
2. Implement JWT authentication
3. Structure a production-ready backend
4. Add request validation and error handling

---

In [ ]:
# Note: FastAPI examples - run with: uvicorn main:app --reload
from typing import Optional, List
from datetime import datetime, timedelta
import hashlib
import json

---
# Section 1: Theory
---

## Why FastAPI?

| Feature | FastAPI | Flask | Django |
|---------|---------|-------|--------|
| Async Support | ✅ Native | ❌ WSGI | ⚠️ Channels |
| Auto Docs | ✅ OpenAPI | ❌ Manual | ❌ Manual |
| Type Hints | ✅ Pydantic | ❌ Optional | ❌ Optional |
| Performance | Very Fast | Moderate | Moderate |

## Key Concepts
- **Async/Await**: Non-blocking I/O for high concurrency
- **Pydantic**: Data validation using Python type hints
- **Dependency Injection**: Clean, testable code structure

---
# Section 2: Hands-On Implementation
---

## 2.1 Basic FastAPI Structure

In [1]:
# main.py structure (conceptual - run in actual file)

fastapi_code = '''
from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel, Field
from typing import Optional, List
import uvicorn

app = FastAPI(
    title="AI Search API",
    description="Semantic search powered by embeddings",
    version="1.0.0"
)

# Pydantic Models
class SearchQuery(BaseModel):
    query: str = Field(..., min_length=1, max_length=500)
    top_k: int = Field(default=5, ge=1, le=20)
    filters: Optional[dict] = None

class SearchResult(BaseModel):
    id: str
    text: str
    score: float
    metadata: Optional[dict] = None

class SearchResponse(BaseModel):
    results: List[SearchResult]
    query_time_ms: float

# Endpoints
@app.get("/")
async def root():
    return {"status": "healthy", "version": "1.0.0"}

@app.post("/search", response_model=SearchResponse)
async def search(query: SearchQuery):
    # Simulate search
    results = [
        SearchResult(id="1", text="Sample result", score=0.95)
    ]
    return SearchResponse(results=results, query_time_ms=15.2)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''
print(fastapi_code)


from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel, Field
from typing import Optional, List
import uvicorn

app = FastAPI(
    title="AI Search API",
    description="Semantic search powered by embeddings",
    version="1.0.0"
)

# Pydantic Models
class SearchQuery(BaseModel):
    query: str = Field(..., min_length=1, max_length=500)
    top_k: int = Field(default=5, ge=1, le=20)
    filters: Optional[dict] = None

class SearchResult(BaseModel):
    id: str
    text: str
    score: float
    metadata: Optional[dict] = None

class SearchResponse(BaseModel):
    results: List[SearchResult]
    query_time_ms: float

# Endpoints
@app.get("/")
async def root():
    return {"status": "healthy", "version": "1.0.0"}

@app.post("/search", response_model=SearchResponse)
async def search(query: SearchQuery):
    # Simulate search
    results = [
        SearchResult(id="1", text="

## 2.2 JWT Authentication

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install passlib[bcrypt]
!pip install python-jose[cryptography]
!pip install hmac


In [7]:
# Simplified JWT implementation
import base64
import hmac
import time
import json
import hashlib



SECRET_KEY = "your-secret-key-here"

def create_jwt(payload: dict, expires_in: int = 3600) -> str:
    """Create a simple JWT token."""
    header = {"alg": "HS256", "typ": "JWT"}
    payload["exp"] = int(time.time()) + expires_in
    
    # Encode header and payload
    header_b64 = base64.urlsafe_b64encode(json.dumps(header).encode()).decode().rstrip("=")
    payload_b64 = base64.urlsafe_b64encode(json.dumps(payload).encode()).decode().rstrip("=")
    
    # Create signature
    message = f"{header_b64}.{payload_b64}"
    signature = hmac.new(SECRET_KEY.encode(), message.encode(), hashlib.sha256).digest()
    signature_b64 = base64.urlsafe_b64encode(signature).decode().rstrip("=")
    
    return f"{header_b64}.{payload_b64}.{signature_b64}"

def verify_jwt(token: str) -> dict:
    """Verify and decode JWT token."""
    parts = token.split(".")
    if len(parts) != 3:
        raise ValueError("Invalid token format")
    
    header_b64, payload_b64, signature_b64 = parts
    
    # Verify signature
    message = f"{header_b64}.{payload_b64}"
    expected_sig = hmac.new(SECRET_KEY.encode(), message.encode(), hashlib.sha256).digest()
    expected_sig_b64 = base64.urlsafe_b64encode(expected_sig).decode().rstrip("=")
    
    if signature_b64 != expected_sig_b64:
        raise ValueError("Invalid signature")
    
    # Decode payload
    payload = json.loads(base64.urlsafe_b64decode(payload_b64 + "=="))
    
    # Check expiration
    if payload.get("exp", 0) < time.time():
        raise ValueError("Token expired")
    
    return payload

In [8]:
# Test JWT
token = create_jwt({"user_id": "123", "role": "admin"})
print(f"Token: {token[:50]}...")

decoded = verify_jwt(token)
print(f"Decoded: {decoded}")

Token: eyJhbGciOiAiSFMyNTYiLCAidHlwIjogIkpXVCJ9.eyJ1c2VyX...
Decoded: {'user_id': '123', 'role': 'admin', 'exp': 1769514118}


## 2.3 Request Validation with Pydantic

In [9]:
from dataclasses import dataclass
from typing import Optional

# Simulating Pydantic-like validation
@dataclass
class DocumentInput:
    text: str
    metadata: Optional[dict] = None
    
    def __post_init__(self):
        if not self.text or len(self.text) < 1:
            raise ValueError("Text cannot be empty")
        if len(self.text) > 10000:
            raise ValueError("Text too long (max 10000 chars)")

# Test validation
try:
    doc = DocumentInput(text="Valid document content")
    print(f"✓ Valid: {doc}")
    
    doc = DocumentInput(text="")
except ValueError as e:
    print(f"✗ Invalid: {e}")

✓ Valid: DocumentInput(text='Valid document content', metadata=None)
✗ Invalid: Text cannot be empty


---
# Section 3: Project Structure
---

In [10]:
project_structure = """
backend/
├── app/
│   ├── __init__.py
│   ├── main.py              # FastAPI app entry point
│   ├── config.py            # Settings & environment
│   ├── dependencies.py      # Dependency injection
│   │
│   ├── api/
│   │   ├── __init__.py
│   │   ├── routes/
│   │   │   ├── search.py    # /search endpoints
│   │   │   ├── documents.py # /documents endpoints
│   │   │   └── auth.py      # /auth endpoints
│   │   └── middleware.py    # CORS, logging, etc.
│   │
│   ├── core/
│   │   ├── embeddings.py    # Embedding generation
│   │   ├── retrieval.py     # Search logic
│   │   └── llm.py           # LLM integration
│   │
│   ├── models/
│   │   ├── schemas.py       # Pydantic models
│   │   └── database.py      # DB models
│   │
│   └── utils/
│       ├── auth.py          # JWT utilities
│       └── logging.py       # Logging setup
│
├── tests/
│   ├── test_search.py
│   └── test_auth.py
│
├── requirements.txt
├── Dockerfile
└── docker-compose.yml
"""
print(project_structure)


backend/
├── app/
│   ├── __init__.py
│   ├── main.py              # FastAPI app entry point
│   ├── config.py            # Settings & environment
│   ├── dependencies.py      # Dependency injection
│   │
│   ├── api/
│   │   ├── __init__.py
│   │   ├── routes/
│   │   │   ├── search.py    # /search endpoints
│   │   │   ├── documents.py # /documents endpoints
│   │   │   └── auth.py      # /auth endpoints
│   │   └── middleware.py    # CORS, logging, etc.
│   │
│   ├── core/
│   │   ├── embeddings.py    # Embedding generation
│   │   ├── retrieval.py     # Search logic
│   │   └── llm.py           # LLM integration
│   │
│   ├── models/
│   │   ├── schemas.py       # Pydantic models
│   │   └── database.py      # DB models
│   │
│   └── utils/
│       ├── auth.py          # JWT utilities
│       └── logging.py       # Logging setup
│
├── tests/
│   ├── test_search.py
│   └── test_auth.py
│
├── requirements.txt
├── Dockerfile
└── docker-compose.yml



---
# Section 4: Unit Tests
---

In [11]:
def run_tests():
    print("Running Unit Tests...\n")
    
    # Test 1: JWT creation
    token = create_jwt({"user": "test"})
    assert len(token.split(".")) == 3
    print("✓ JWT creation test passed")
    
    # Test 2: JWT verification
    decoded = verify_jwt(token)
    assert decoded["user"] == "test"
    print("✓ JWT verification test passed")
    
    # Test 3: Document validation
    doc = DocumentInput(text="Valid text")
    assert doc.text == "Valid text"
    print("✓ Document validation test passed")
    
    # Test 4: Invalid document
    try:
        DocumentInput(text="")
        assert False
    except ValueError:
        print("✓ Invalid document rejection test passed")
    
    print("\n🎉 All tests passed!")

run_tests()

Running Unit Tests...

✓ JWT creation test passed
✓ JWT verification test passed
✓ Document validation test passed
✓ Invalid document rejection test passed

🎉 All tests passed!


---
# Section 5: Interview Prep
---

### Q1: Why use async/await in FastAPI?
**Answer:** Non-blocking I/O allows handling many concurrent requests. While waiting for DB/API calls, the event loop serves other requests.

### Q2: How do you handle authentication in APIs?
**Answer:** JWT tokens in Authorization header. Stateless, scalable. Use refresh tokens for long sessions.

### Q3: What is dependency injection in FastAPI?
**Answer:** `Depends()` function provides reusable components (DB sessions, auth, etc.) to endpoints. Improves testability.

---
# Section 6: Deliverable
---

**Created:** `backend_skeleton.py` with auth, search endpoints, validation

**Next Week:** Vector DB Integration (PostgreSQL + pgvector)